In [12]:
import sys

# Add the parent directory to the system path
sys.path.append(r'C:\Users\kaise\OneDrive\Desktop\Codebase\FYP\utils')

In [13]:
import re
import requests
import geopandas as gpd
import numpy as np
import pandas as pd
from read_df_from_s3 import read_df_from_s3
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

date = "2024-01-26"
df = read_df_from_s3(f"rental_prices/ninety_nine/processed_v2/{date}.parquet.gzip")
print(f"Dataframe downloaded with shape {df.shape}")

Dataframe downloaded with shape (2748, 22)


In [2]:
df.head()

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,price/sqft,floor_level,...,built_year,tenure,property_type,url,facilities,building_name,nearest_mrt,distance_to_nearest_mrt,latitude,longitude
0,2 Bed Condo in Marina One Residences,4dNy8WuJGt9TknKjKJ4DXy,Boat Quay / Raffles Place / Marina,"$7,400/mo",2 Beds,2 Baths,"1,176 sqft",23 Marina Way,$6.29 psf,None,...,2018.0,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/mari...,"['Pool Deck', 'Hydrotherapy Pool', 'Wading Poo...",Marina One Residences,Marina Bay,219.281,1.27671498507474,103.853326932298
1,1 Bed Condo in The Sail @ Marina Bay,oQroUcBdTUjvUSMTYKtiZg,Boat Quay / Raffles Place / Marina,"$4,790/mo",1 Bed,1 Bath,667 sqft,4 Marina Boulevard,$7.18 psf,High,...,2008.0,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/the-...,"['Sky Terrace', 'Washer', 'Steam Room', 'Oven'...",The Sail @ Marina Bay,Downtown,149.256,1.28076943504219,103.852658562152
2,2 Bed Condo in V On Shenton,LZvu3LEYKKJ2RQgcthrtcn,Boat Quay / Raffles Place / Marina,"$6,500/mo",2 Beds,2 Baths,"1,055 sqft",5A Shenton Way,$6.16 psf,Mid,...,2017.0,99-year leasehold,Condo Whole Unit,https://www.99.co/singapore/rent/property/v-on...,"['Spa Pool', 'Aircon', 'Fitness Corner', 'Fibr...",V On Shenton,Shenton Way,171.428,1.27708340707631,103.849181343548
3,Studio Condo in V On Shenton,MWASUKzWmFZWRKsnERisRK,Boat Quay / Raffles Place / Marina,"$3,388/mo",Studio,1 Bath,474 sqft,5A Shenton Way,$7.15 psf,Low,...,2017.0,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/v-on...,"['Spa Pool', 'Underwater Fitness Station', 'Ou...",V On Shenton,Shenton Way,171.428,1.27708340707631,103.849181343548
4,4 Bed Condo in The Sail @ Marina Bay,GsmAMQH99u3HkSxnAzaTQV,Boat Quay / Raffles Place / Marina,"$11,999/mo",4 Beds,3 Baths,"2,077 sqft",2 Marina Boulevard,$5.78 psf,High,...,2008.0,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/the-...,"['Jogging Track', 'Tennis Court', 'Fun Pool', ...",The Sail @ Marina Bay,Downtown,149.256,1.28076943504219,103.852658562152


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   property_name            2748 non-null   object 
 1   listing_id               2748 non-null   object 
 2   district                 2748 non-null   object 
 3   price                    2748 non-null   object 
 4   bedroom                  2748 non-null   object 
 5   bathroom                 2398 non-null   object 
 6   dimensions               2748 non-null   object 
 7   address                  2748 non-null   object 
 8   price/sqft               2748 non-null   object 
 9   floor_level              1896 non-null   object 
 10  furnishing               2629 non-null   object 
 11  facing                   601 non-null    object 
 12  built_year               2666 non-null   float64
 13  tenure                   2685 non-null   object 
 14  property_type           

In [4]:
df.isna().sum()

property_name                 0
listing_id                    0
district                      0
price                         0
bedroom                       0
bathroom                    350
dimensions                    0
address                       0
price/sqft                    0
floor_level                 852
furnishing                  119
facing                     2147
built_year                   82
tenure                       63
property_type                 0
url                           0
facilities                    0
building_name                 0
nearest_mrt                 166
distance_to_nearest_mrt     166
latitude                    166
longitude                   166
dtype: int64

Duplicate listing IDs -> Only leave the one with the correct district


In [5]:
len(df[df.duplicated(subset='listing_id', keep=False)])

2195

Problem with the **address** column (Typically HDBs / Landed) -> Replace with the building name


In [6]:
all = df[df['address'].str.contains('For Rent', case=False)]
len(all)

632

Missing coordinates -> Retrieve with OneMap API


In [7]:
df[df['latitude'].isna() | df['longitude'].isna()]

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,price/sqft,floor_level,...,built_year,tenure,property_type,url,facilities,building_name,nearest_mrt,distance_to_nearest_mrt,latitude,longitude
40,6 Bed House on Wilkinson Road,R3wC4GWBFKGuynWHef7afU,Boat Quay / Raffles Place / Marina,"$15,000/mo",6 Beds,7 Baths,"7,200 sqft (built-up)",Landed House for Rent,$2.08 psf,None,...,NaN,None,Semi-Detached House,https://www.99.co/singapore/rent/property/wilk...,"['Renovated', 'High Ceiling', 'Fibre Ready', '...",6 Bed House on Wilkinson Road,None,NaN,None,None
54,5 Bed House on Cluny Park Road,m82ubNLmdiWhH4mCaC4vLe,Boat Quay / Raffles Place / Marina,"$78,000/mo",5 Beds,7 Baths,"7,736 sqft (built-up)",Landed House for Rent,$10.08 psf,None,...,NaN,None,Good Class Bungalow,https://www.99.co/singapore/rent/property/clun...,"['Fridge', 'Maid Room', 'Aircon', 'Stove', 'Wa...",5 Bed House on Cluny Park Road,None,NaN,None,None
80,2 Bed Condo in D'ixoras,kA9bRLSr5X93z6MkCg4CT5,Boat Quay / Raffles Place / Marina,"$4,600/mo",2 Beds,2 Baths,840 sqft,23 Minbu Road,$5.48 psf,High,...,2013.0,Freehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/dixo...,"['Parking', 'Bomb Shelter', 'BBQ', 'Swimming P...",D'ixoras,None,NaN,None,None
144,4 Bed House on Alana,8dWpH9r8Y4GE62Lmc8JtPG,Chinatown / Tanjong Pagar,"$8,700/mo",4 Beds,4 Baths,"3,057 sqft (built-up)",Sunrise Terrace,$2.85 psf,None,...,NaN,103-year leasehold,Terraced House,https://www.99.co/singapore/rent/property/sunr...,"['Pavilion', 'Spa Pool', 'Pool Deck', 'Patio /...",4 Bed House on Alana,None,NaN,None,None
154,5 Bed House on Shamrock Park,XCTfCbw3axroufTVW37Fsm,Chinatown / Tanjong Pagar,"$25,000/mo",5 Beds,6 Baths,"7,000 sqft (built-up)",Namly Grove,$3.57 psf,None,...,2008.0,Freehold,Bungalow,https://www.99.co/singapore/rent/property/naml...,"['Maid Room', 'Fibre Ready', 'High Ceiling', '...",5 Bed House on Shamrock Park,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,4 Bed House on Seletar Hills Estate,mKYhXTDbgK7PGCxdx3g9fd,Seletar / Yio Chu Kang,"$6,500/mo",4 Beds,5 Baths,"3,336 sqft (built-up)",Seletar Hills Drive,$1.95 psf,None,...,NaN,None,Cluster House,https://www.99.co/singapore/rent/property/sele...,"['Maid Room', 'Corner Unit', 'Bomb Shelter']",4 Bed House on Seletar Hills Estate,None,NaN,None,None
2677,5 Bed House on Este Villa,WpEWyqfwYxpm3DwgEqsmtn,Seletar / Yio Chu Kang,"$7,800/mo",5 Beds,5 Baths,"3,359 sqft (built-up)",Nim Road,$2.32 psf,None,...,2013.0,Freehold,Cluster House,https://www.99.co/singapore/rent/property/nim-...,"['Patio / PES', 'Clubhouse', 'Maid Room', 'Swi...",5 Bed House on Este Villa,None,NaN,None,None
2679,5 Bed House on Seletar Park,kXTMyPoJseLDoDcAZBiJWT,Seletar / Yio Chu Kang,"$8,000/mo",5 Beds,5 Baths,"2,500 sqft (built-up)",Landed House for Rent,$3.2 psf,None,...,1986.0,999-year leasehold,Semi-Detached House,https://www.99.co/singapore/rent/property/prop...,[],5 Bed House on Seletar Park,None,NaN,None,None
2680,5 Bed House on Seletar Park,XXv6kUVJwmQHhu7FeYgWtc,Seletar / Yio Chu Kang,"$9,300/mo",5 Beds,5 Baths,"3,000 sqft (built-up)",Landed House for Rent,$3.1 psf,None,...,1986.0,999-year leasehold,Semi-Detached House,https://www.99.co/singapore/rent/property/prop...,[],5 Bed House on Seletar Park,None,NaN,None,None


Convert to categorical


In [8]:
df['floor_level'].unique()

array([None, 'High', 'Mid', 'Low', 'High (42 total)', 'Ground',
       'Penthouse', 'Top', 'High (15 total)', 'High (44 total)',
       'Mid (18 total)', 'Low (8 total)', 'High (31 total)',
       'Top (10 total)', 'High (19 total)'], dtype=object)

In [9]:
df['facing'].unique()

array(['South East', 'South West', 'South', None, 'North', 'North East',
       'East', 'North West', 'West'], dtype=object)

In [10]:
df['furnishing'].unique()

array(['Partial', 'Fully', None, 'Unfurnished', 'Flexible'], dtype=object)

In [11]:
df['tenure'].unique()

array(['99-year leasehold', None, 'Freehold', '999-year leasehold',
       '103-year leasehold', '956-year leasehold', '929-year leasehold',
       '946-year leasehold', '60-year leasehold', '100-year leasehold'],
      dtype=object)

In [12]:
df['property_type'].unique()

array(['Apartment Whole Unit ', 'Condo Whole Unit ', 'Apartment', 'Condo',
       'HDB 5 Rooms', 'Semi-Detached House', 'HDB', 'Corner Terrace ',
       'Executive Condo Whole Unit ', 'Good Class Bungalow ',
       'Executive Condo', 'HDB 3 Rooms', 'HDB 4 Rooms', 'HDB 2 Rooms',
       'Terraced House ', 'Bungalow ', 'HDB Executive', 'Shophouse ',
       'Land Only', 'Conservation House ', 'Cluster House ', 'Townhouse '],
      dtype=object)

# Data Preprocessing


Add **building_name** column


In [13]:
df['building_name'] = df['property_name'].apply(lambda x: x.split(" in ")[-1])
len(df['building_name'].unique())

640

Fix **address** column


In [14]:
indexes = df.loc[df['address'].str.contains(
    'Landed House For Rent', case=False)].index
df.loc[indexes, 'address'] = df.loc[indexes,
                                    'building_name'].apply(lambda x: [s.strip() for s in re.split(r'\bon\b|\bin\b', x) if s.strip()][-1])
df.loc[indexes, ['address', 'property_name', 'building_name']]

,address,property_name,building_name
40,Wilkinson Road,6 Bed House on Wilkinson Road,6 Bed House on Wilkinson Road
54,Cluny Park Road,5 Bed House on Cluny Park Road,5 Bed House on Cluny Park Road
73,Jalan Kembangan,Master Room in Jalan Kembangan,Jalan Kembangan
145,Jalan Raya,Common Room in Jalan Raya,Jalan Raya
181,Lim Tai See Walk,5 Bed House on Lim Tai See Walk,5 Bed House on Lim Tai See Walk
265,Ceylon Road,6 Bed House on Ceylon Road,6 Bed House on Ceylon Road
469,Jalan Senyum,6 Bed House on Jalan Senyum,6 Bed House on Jalan Senyum
506,Armenian Street,Studio House on Armenian Street,Studio House on Armenian Street
559,Yio Chu Kang Road,Master Room in Yio Chu Kang Road,Yio Chu Kang Road
596,Jalan Senyum,6 Bed House on Jalan Senyum,6 Bed House on Jalan Senyum


In [15]:
indexes = df.loc[df['address'].str.contains('For Rent', case=False)].index
df.loc[indexes, 'address'] = df.loc[indexes, 'building_name']
df.loc[indexes, ['address', 'property_name', 'building_name']]

,address,property_name,building_name
39,316 Sembawang Vista,5 Room HDB in 316 Sembawang Vista,316 Sembawang Vista
45,18D Holland Drive,Common Room in 18D Holland Drive,18D Holland Drive
47,194 Rivervale Drive,Common Room in 194 Rivervale Drive,194 Rivervale Drive
50,643 Punggol Central,5 Room HDB in 643 Punggol Central,643 Punggol Central
55,163 Simei Road,5 Room HDB in 163 Simei Road,163 Simei Road
...,...,...,...
2723,89 Bedok North Street 4,4 Room HDB in 89 Bedok North Street 4,89 Bedok North Street 4
2725,425 Choa Chu Kang Avenue 4,Common Room in 425 Choa Chu Kang Avenue 4,425 Choa Chu Kang Avenue 4
2727,5 Marine Terrace,3 Room HDB in 5 Marine Terrace,5 Marine Terrace
2736,338D Anchorvale Crescent,5 Room HDB in 338D Anchorvale Crescent,338D Anchorvale Crescent


In [16]:
building_map = {}
for building_name, group in df.groupby('building_name'):
    for _, row in group.iterrows():
        if pd.isna(row['latitude']) or pd.isna(row['longitude']):
            continue

        building_map[building_name] = (row['latitude'], row['longitude'])
        break
print(building_map)

{'103B Bidadari Park Drive': ('1.33508945420373', '103.871337279805'), '106 Jurong East Street 13': ('1.33724965052823', '103.737674650927'), '106A Depot Road': ('1.28061364977894', '103.810967870883'), '107 Yishun Ring Road': ('1.43221807625653', '103.827520202898'), '108 Potong Pasir Avenue 1': ('1.33472004322162', '103.867928446438'), '108C Mcnair Road': ('1.31984417513752', '103.859462494536'), '113 Clementi Street 13': ('1.32368401078529', '103.77013234931'), '116 Clementi Street 13': ('1.32411253933745', '103.771925036203'), '116 Lorong 2 Toa Payoh': ('1.34047287241805', '103.846142516335'), '116A Rivervale Drive': ('1.38246038121425', '103.902238675198'), '117 Ang Mo Kio Avenue 4': ('1.3737170825368', '103.835610469339'), '118 Alkaff Crescent': ('1.33731461799125', '103.873879971632'), '120 Grange': ('1.29997909930352', '103.825522814492'), '121 Ang Mo Kio Avenue 3': ('1.36970563478325', '103.843515981311'), '122A Edgedale Plains': ('1.39350981217359', '103.909106562659'), '127C

In [17]:
def fetch_coordinates(location_name):
    url = 'https://www.onemap.gov.sg/api/common/elastic/search'
    params = {
        'searchVal': location_name,
        'returnGeom': 'Y',
        'getAddrDetails': 'Y',
        'pageNum': 1
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['found'] > 0:
            return location_name, (data['results'][0]['LATITUDE'], data['results'][0]['LONGITUDE'])
    print(f"Failed to fetch coordinates for {location_name}")
    return location_name, (np.nan, np.nan)


def update_lat_long_v1(df, building_map):
    # Locate rows with null lat and long
    df_null_coords = df[(df['latitude'].isnull()) & (df['longitude'].isnull())]
    building_names = df_null_coords['building_name'].unique()

    building_names_duplicate = [
        name for name in building_names if name in building_map]
    for building_name in building_names_duplicate:
        df.loc[df['building_name'] == building_name,
               'latitude'] = building_map[building_name][0]
        df.loc[df['building_name'] == building_name,
               'longitude'] = building_map[building_name][1]

    building_names_to_fetch = [
        name for name in building_names if name not in building_map]
    with ThreadPoolExecutor() as executor:
        print(f"Using {executor._max_workers} threads to fetch coordinates\n")
        future_to_coords = {executor.submit(
            fetch_coordinates, name): name for name in building_names_to_fetch}
        for future in as_completed(future_to_coords):
            result = future.result()
            if result is None:
                continue

            building_name, coords = result
            building_map[building_name] = coords
            df.loc[df['building_name'] ==
                   building_name, 'latitude'] = coords[0]
            df.loc[df['building_name'] ==
                   building_name, 'longitude'] = coords[1]

    return df, building_map


def update_lat_long_v2(df):
    # Locate rows with null lat and long
    df_null_coords = df[(df['latitude'].isnull()) & (df['longitude'].isnull())]
    addresses = df_null_coords['address'].unique()

    with ThreadPoolExecutor() as executor:
        print(f"Using {executor._max_workers} threads to fetch coordinates\n")
        future_to_coords = {executor.submit(
            fetch_coordinates, name): name for name in addresses}
        for future in as_completed(future_to_coords):
            result = future.result()
            if result is None:
                continue

            address, coords = result
            df.loc[df['address'] ==
                   address, 'latitude'] = coords[0]
            df.loc[df['address'] ==
                   address, 'longitude'] = coords[1]

    return df


df, building_map = update_lat_long_v1(df, building_map)

Using 12 threads to fetch coordinates



Failed to fetch coordinates for 3 Bed House on Kimsia Park
Failed to fetch coordinates for 6 Bed House on Ceylon Road
Failed to fetch coordinates for 4 Bed House on Alana
Failed to fetch coordinates for Pollen & Bleu
Failed to fetch coordinates for 5 Bed House on Lim Tai See Walk
Failed to fetch coordinates for D'ixoras
Failed to fetch coordinates for 1953
Failed to fetch coordinates for 6 Bed House on Wilkinson Road
Failed to fetch coordinates for 5 Bed House on Shamrock Park
Failed to fetch coordinates for 5 Bed House on Cluny Park Road
Failed to fetch coordinates for 4 Bed House on Teacher's Housing Estate
Failed to fetch coordinates for 2 Bed House on Macpherson Garden Estate
Failed to fetch coordinates for The Woodleigh Residences / The Woodleigh Mall
Failed to fetch coordinates for Hillion Residences / Hillion Mall
Failed to fetch coordinates for 6 Bed House on Sentosa Cove
Failed to fetch coordinates for The Brooks I & II
Failed to fetch coordinates for Nassim Ville
Failed to fe

In [18]:
# Fetch using addresses next
df = update_lat_long_v2(df)

Using 12 threads to fetch coordinates



In [19]:
from location_constants import DISTRICTS
REVERSE_DISTRICTS = {v: k for k, v in DISTRICTS.items()}

In [20]:
df["district_id"] = df["district"].map(REVERSE_DISTRICTS)

In [21]:
from motherduckdb_connector import connect_to_motherduckdb
from shapely import wkt

db = connect_to_motherduckdb()
geometry_df = db.query_df("SELECT * FROM plan_area_mapping")
geometry_df['polygon'] = geometry_df['polygon'].apply(wkt.loads)
geometry_df = gpd.GeoDataFrame(geometry_df, geometry='polygon')
db.close()

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘



In [22]:
from location_constants import PLAN_AREA_MAPPING


def get_district(lat, long):
    # swap long and la
    point = Point(long, lat)
    plan_area = geometry_df.loc[geometry_df.contains(
        point), 'plan_area'].squeeze()

    if isinstance(plan_area, pd.Series):
        if plan_area.empty:
            return ""
        plan_area = plan_area.mode()

    return PLAN_AREA_MAPPING[plan_area]

In [23]:
df["tmp_district_id"] = df.apply(
    lambda x: get_district(x["latitude"], x["longitude"]), axis=1)
df.drop(df[(df.duplicated(subset='listing_id', keep=False)) & (
    df["district_id"] != df["tmp_district_id"])].index, inplace=True)
df.drop(columns=["tmp_district_id"], inplace=True)

Now we are left with the real duplicates


In [24]:
df[df.duplicated(subset='listing_id', keep=False)].sort_values(by='listing_id')

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,price/sqft,floor_level,...,tenure,property_type,url,facilities,building_name,nearest_mrt,distance_to_nearest_mrt,latitude,longitude,district_id
1164,Common Room in 73 Lorong 4 Toa Payoh,2eRXwDnSTb8jPYetFXkSQg,Balestier / Toa Payoh,"$1,200/mo",Common Room,None,160 sqft (room),73 Lorong 4 Toa Payoh,$7.5 psf,High,...,99-year leasehold,HDB 3 Rooms,https://www.99.co/singapore/rooms/property/73-...,['Renovated'],73 Lorong 4 Toa Payoh,Toa Payoh,511.039,1.33432001132262,103.851714329027,D12
1115,Common Room in 73 Lorong 4 Toa Payoh,2eRXwDnSTb8jPYetFXkSQg,Balestier / Toa Payoh,"$1,200/mo",Common Room,None,160 sqft (room),73 Lorong 4 Toa Payoh,$7.5 psf,High,...,99-year leasehold,HDB 3 Rooms,https://www.99.co/singapore/rooms/property/73-...,['Renovated'],73 Lorong 4 Toa Payoh,Toa Payoh,511.039,1.33432001132262,103.851714329027,D12
415,2 Bed Condo in Hundred Trees,3q8vivHJASSNPY6SmpsShA,Buona Vista / West Coast / Clementi,"$4,500/mo",2 Beds,2 Baths,786 sqft,95 West Coast Drive,$5.73 psf,High,...,956-year leasehold,Condo Whole Unit,https://www.99.co/singapore/rent/property/hund...,"['Parking', 'Pond', 'Tennis Court', 'Lounge', ...",Hundred Trees,Clementi,799.153,1.3155055432603,103.758056882156,D05
487,2 Bed Condo in Hundred Trees,3q8vivHJASSNPY6SmpsShA,Buona Vista / West Coast / Clementi,"$4,500/mo",2 Beds,2 Baths,786 sqft,95 West Coast Drive,$5.73 psf,High,...,956-year leasehold,Condo Whole Unit,https://www.99.co/singapore/rent/property/hund...,"['Parking', 'Pond', 'Tennis Court', 'Lounge', ...",Hundred Trees,Clementi,799.153,1.3155055432603,103.758056882156,D05
0,2 Bed Condo in Marina One Residences,4dNy8WuJGt9TknKjKJ4DXy,Boat Quay / Raffles Place / Marina,"$7,400/mo",2 Beds,2 Baths,"1,176 sqft",23 Marina Way,$6.29 psf,None,...,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/mari...,"['Pool Deck', 'Hydrotherapy Pool', 'Wading Poo...",Marina One Residences,Marina Bay,219.281,1.27671498507474,103.853326932298,D01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,Master Room in 230 Bukit Batok East Avenue 3,nmmwN7A4yUbvYiQHcCcwWu,Bukit Batok / Bukit Panjang / Choa Chu Kang,"$1,250/mo",Master Room,1 Bath,160 sqft (room),230 Bukit Batok East Avenue 3,$7.81 psf,Mid,...,99-year leasehold,HDB,https://www.99.co/singapore/rooms/property/230...,"['Attached Bathroom', 'Wireless Internet', 'Ai...",230 Bukit Batok East Avenue 3,Bukit Batok,622.791,1.34771540923723,103.754994802909,D23
1721,1 Bed Condo in Coco Palms,oBgXdJcytLwfvjKXE9LBmi,Pasir Ris / Tampines,"$3,200/mo",1 Bed,1 Bath,463 sqft,21 Pasir Ris Grove,$6.91 psf,Mid,...,99-year leasehold,Condo Whole Unit,https://www.99.co/singapore/rent/property/coco...,"['Jogging Track', 'Tennis Court', 'Fun Pool', ...",Coco Palms,Pasir Ris,466.163,1.37150158736384,103.945323459304,D18
1792,1 Bed Condo in Coco Palms,oBgXdJcytLwfvjKXE9LBmi,Pasir Ris / Tampines,"$3,200/mo",1 Bed,1 Bath,463 sqft,21 Pasir Ris Grove,$6.91 psf,Mid,...,99-year leasehold,Condo Whole Unit,https://www.99.co/singapore/rent/property/coco...,"['Jogging Track', 'Tennis Court', 'Fun Pool', ...",Coco Palms,Pasir Ris,466.163,1.37150158736384,103.945323459304,D18
79,1 Bed Condo in The Sail @ Marina Bay,oQroUcBdTUjvUSMTYKtiZg,Boat Quay / Raffles Place / Marina,"$4,790/mo",1 Bed,1 Bath,667 sqft,4 Marina Boulevard,$7.18 psf,High,...,99-year leasehold,Apartment Whole Unit,https://www.99.co/singapore/rent/property/the-...,"['Sky Terrace', 'Washer', 'Steam Room', 'Oven'...",The Sail @ Marina Bay,Downtown,149.256,1.28076943504219,103.852658562152,D01


In [25]:
df.drop_duplicates(subset='listing_id', keep='first', inplace=True)

In [26]:
from find_closest import find_nearest, fetch_mrt_info


def update_mrt(df):
    df_null_mrt = df[df["nearest_mrt"].isnull()]

    mrt_info = fetch_mrt_info()
    df_null_mrt = find_nearest(
        df_null_mrt, mrt_info, "nearest_mrt", "distance_to_nearest_mrt")
    df.update(df_null_mrt)

    return df


df = update_mrt(df)

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘

  station_name   longitude  latitude station_code   color         line
0    Esplanade  103.855612  1.293260          CC3  Orange       Circle
1   Paya Lebar  103.892272  1.317369          EW8   Green    East West
2   Paya Lebar  103.892272  1.317369          CC9  Orange       Circle
3  Dhoby Ghaut  103.845833  1.299044         NS24     Red  North South
4  Dhoby Ghaut  103.845833  1.299044          NE6  Purple   North East



In [27]:
from find_closest import fetch_hawker_info

df["nearest_hawker"] = None
df["distance_to_nearest_hawker"] = float("inf")


def update_hawker(df):
    df_null_hawker = df[df["nearest_hawker"].isnull()]

    hawker_info = fetch_hawker_info()
    df_null_hawker = find_nearest(
        df_null_hawker, hawker_info, "nearest_hawker", "distance_to_nearest_hawker")
    df.update(df_null_hawker)

    return df


df = update_hawker(df)

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘

   hawker_id                             name  \
0          1      Market Street Hawker Centre   
1          2     Marsiling Mall Hawker Centre   
2          3     Margaret Drive Hawker Centre   
3          4  Fernvale Hawker Centre & Market   
4          5        One Punggol Hawker Centre   

                                     building_name           street_name  \
0                                     CapitaSpring         Market Street   
1                                   Marsiling Mall   Woodlands Street 12   
2                                                         Margaret Drive   
3  Fernvale Community Club, Hawker Centre & Market

In [28]:
from find_closest import fetch_supermarket_info

df["nearest_supermarket"] = None
df["distance_to_nearest_supermarket"] = float("inf")


def update_supermarket(df):
    df_null_supermarket = df[df["nearest_supermarket"].isnull()]

    supermarket_info = fetch_supermarket_info()
    df_null_supermarket = find_nearest(
        df_null_supermarket, supermarket_info, "nearest_supermarket", "distance_to_nearest_supermarket")
    df.update(df_null_supermarket)

    return df


df = update_supermarket(df)

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘

   supermarket_id                                         name  \
0               1  LI LI CHENG SUPERMARKET (PUNGGOL) PTE. LTD.   
1               2              SHENG SIONG SUPERMARKET PTE LTD   
2               3        COLD STORAGE SINGAPORE (1983) PTE LTD   
3               4        COLD STORAGE SINGAPORE (1983) PTE LTD   
4               5                      YES SUPERMARKET PTE LTD   

            street_name postal_code   longitude  latitude  
0         PUNGGOL PLACE      823273  103.901262  1.402303  
1  UPPER BOON KENG ROAD      380011  103.870914  1.314239  
2      HOUGANG AVENUE 8      530683  103.886366  1.373321  
3  BEDOK RES

In [29]:
from find_closest import fetch_primary_school_info

df["nearest_sch"] = None
df["distance_to_nearest_sch"] = float("inf")


def update_primary_school(df):
    df_null_sch = df[df["nearest_sch"].isnull()]

    sch_info = fetch_primary_school_info()
    df_null_sch = find_nearest(
        df_null_sch, sch_info, "nearest_sch", "distance_to_nearest_sch")
    df.update(df_null_sch)

    return df


df = update_primary_school(df)

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘

   school_id                          name         area   longitude  latitude
0          1      Admiralty Primary School    Woodlands  103.800214  1.442550
1          2  Ahmad Ibrahim Primary School       Yishun  103.832942  1.433153
2          3                Ai Tong School       Bishan  103.833020  1.360583
3          4      Alexandra Primary School  Bukit Merah  103.824425  1.291334
4          5   Anchor Green Primary School     Sengkang  103.887165  1.390370



In [30]:
from find_closest import fetch_mall_info

df["nearest_mall"] = None
df["distance_to_nearest_mall"] = float("inf")


def update_mall(df):
    df_null_mall = df[df["nearest_mall"].isnull()]

    mall_info = fetch_mall_info()
    df_null_mall = find_nearest(
        df_null_mall, mall_info, "nearest_mall", "distance_to_nearest_mall")
    df.update(df_null_mall)

    return df


df = update_mall(df)

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘

   mall_id                            name   longitude  latitude
0        1                          100 AM  103.843471  1.274588
1        2                    313@Somerset  103.838361  1.301014
2        3                          Aperia  103.863930  1.310867
3        4  Balestier Hill Shopping Centre  103.842572  1.325596
4        5                      Bugis Cube  103.855635  1.298141



In [31]:
def simplify_lease_type(lease_type):
    if pd.isnull(lease_type):
        return None
    elif 'leasehold' in lease_type:
        return 'leasehold'
    else:
        return 'freehold'


def simplify_property_type(property_type):
    if pd.isnull(property_type):
        return None
    elif 'Condo' in property_type and 'Executive' not in property_type:
        return 'Condo'
    elif 'HDB' in property_type and 'Executive' not in property_type:
        return 'HDB'
    elif 'Apartment' in property_type and 'Executive' not in property_type:
        return 'Apartment'
    elif 'Bungalow' in property_type:
        return 'HDB'
    elif 'Land' in property_type:
        return 'Landed'

    return property_type.strip()


df['property_type'] = df['property_type'].apply(
    simplify_property_type).astype('category')
df['furnishing'] = df['furnishing'].fillna('Unfurnished').astype('category')
df['facing'] = df['facing'].astype('category')
df['tenure'] = df['tenure'].apply(simplify_lease_type).astype('category')
df['floor_level'] = df['floor_level'].str.replace(
    r'\s*\(\d+ total\)', '', regex=True).astype('category')
df['district'] = df['district'].astype('category')

In [32]:
def extract_num_price(x):
    if not x:
        return ""
    res = re.findall(r'\d[\d,]*', x)
    return res[0] if res else ""


def extract_num_bedroom(x):
    if not x:
        return "0"
    res = re.findall(r'\d[\d,]*', x)
    if not res:
        return "1"
    return res[0]


def extract_num(x):
    if not x:
        return None
    res = re.findall(r'\d[\d,]*', x)
    return res[0] if res else None


df['price'] = df['price'].apply(
    extract_num_price).str.replace(',', '').astype(int)
df['bedroom'] = df['bedroom'].apply(
    extract_num_bedroom).astype(int)
df['bathroom'] = df['bathroom'].apply(
    extract_num).fillna("0").astype(int)
df['dimensions'] = df['dimensions'].apply(
    extract_num).str.replace(',', '').astype(int)
df['price/sqft'] = df['price/sqft'].apply(
    extract_num).astype(float)
df['built_year'] = df['built_year'].fillna(9999).astype(int)

In [33]:
df["is_whole_unit"] = True

In [34]:
def update_room_rental_properties(df):
    indexes = df.loc[(df['property_name'].str.contains(
        'Room', case=False))].index
    df.loc[indexes, 'is_whole_unit'] = False
    df.loc[indexes, 'bedroom'] = 1
    df.loc[indexes, 'bathroom'] = 0

    indexes = df.loc[(df['property_name'].str.contains(
        'Studio', case=False))].index
    df.loc[indexes, 'bedroom'] = 1

    return df


df = update_room_rental_properties(df)

In [35]:
df['has_pool'] = df['facilities'].apply(lambda x: 'pool' in x.lower())
df['has_gym'] = df['facilities'].apply(lambda x: 'gym' in x.lower())

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 2682
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   property_name                    832 non-null    object  
 1   listing_id                       832 non-null    object  
 2   district                         832 non-null    category
 3   price                            832 non-null    int32   
 4   bedroom                          832 non-null    int32   
 5   bathroom                         832 non-null    int32   
 6   dimensions                       832 non-null    int32   
 7   address                          832 non-null    object  
 8   price/sqft                       832 non-null    float64 
 9   floor_level                      499 non-null    category
 10  furnishing                       832 non-null    category
 11  facing                           101 non-null    category
 12  built_y

In [37]:
df["fingerprint"] = df["listing_id"] + "-" + df["price"].astype(str)
df["fingerprint"]

0        4dNy8WuJGt9TknKjKJ4DXy-7400
1        oQroUcBdTUjvUSMTYKtiZg-4790
2        LZvu3LEYKKJ2RQgcthrtcn-6500
3        MWASUKzWmFZWRKsnERisRK-3388
4       GsmAMQH99u3HkSxnAzaTQV-11999
                    ...             
2678     EgTLH9sz9SZJKZHBeghZqG-4500
2679     kXTMyPoJseLDoDcAZBiJWT-8000
2680     XXv6kUVJwmQHhu7FeYgWtc-9300
2681      c3crMegeUi6mVAgnqyHbTf-550
2682     jBSMa2qPVipCR9qEYQxajx-3500
Name: fingerprint, Length: 832, dtype: object

In [38]:
df['source'] = 'ninety_nine'
df['scraped_on'] = datetime.strptime(date, '%Y-%m-%d')
df['last_updated'] = df['scraped_on']

df.rename(columns={
    'price/sqft': 'price_per_sqft',
    'distance_to_nearest_mrt': 'distance_to_mrt_in_m',
    'distance_to_nearest_hawker': 'distance_to_hawker_in_m',
    'distance_to_nearest_sch': 'distance_to_sch_in_m',
    'distance_to_nearest_supermarket': 'distance_to_supermarket_in_m',
    'distance_to_nearest_mall': 'distance_to_mall_in_m',
}, inplace=True)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 2682
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   property_name                 832 non-null    object        
 1   listing_id                    832 non-null    object        
 2   district                      832 non-null    category      
 3   price                         832 non-null    int32         
 4   bedroom                       832 non-null    int32         
 5   bathroom                      832 non-null    int32         
 6   dimensions                    832 non-null    int32         
 7   address                       832 non-null    object        
 8   price_per_sqft                832 non-null    float64       
 9   floor_level                   499 non-null    category      
 10  furnishing                    832 non-null    category      
 11  facing                        1

In [40]:
df.describe()

,price,bedroom,bathroom,dimensions,price_per_sqft,built_year,distance_to_mrt_in_m,distance_to_hawker_in_m,distance_to_supermarket_in_m,distance_to_sch_in_m,distance_to_mall_in_m
count,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000
mean,4799.891827,2.038462,1.542067,1056.137019,5.627404,2399.954327,648.635965,884.318203,439.223591,693.237751,784.844268
std,4371.356121,1.214022,1.462951,1055.995856,4.151536,1731.159609,463.811561,708.309635,409.594941,474.326586,630.521782
min,500.000000,1.000000,0.000000,10.000000,0.000000,1970.000000,54.012000,40.548000,0.039000,66.069000,0.030000
25%,2900.000000,1.000000,0.000000,461.750000,4.000000,1998.000000,314.410750,409.594750,190.350000,386.241750,333.230250
50%,4200.000000,2.000000,1.000000,829.000000,5.000000,2011.000000,527.407500,659.308000,344.168000,588.247500,639.493500
75%,5799.250000,3.000000,2.000000,1250.250000,7.000000,2018.000000,830.122000,1085.306000,542.080000,838.580500,1083.197750
max,78000.000000,7.000000,7.000000,8500.000000,90.000000,9999.000000,3536.145000,3742.359000,3318.827000,3309.225000,3462.201000


In [41]:
from location_constants import PROPERTY_LISTING_COLS, RENTAL_PRICE_HISTORY_COLS
df = df[PROPERTY_LISTING_COLS]

In [42]:
from motherduckdb_connector import connect_to_motherduckdb
db = connect_to_motherduckdb()
existing = db.query_df(
    "SELECT listing_id, fingerprint, last_updated FROM property_listing")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ hawker_centre_info   │
│ mall_info            │
│ mrt_info             │
│ plan_area_mapping    │
│ primary_school_info  │
│ property_listing     │
│ rental_price_history │
│ supermarket_info     │
│ tmp                  │
└──────────────────────┘



In [43]:
df = df.merge(existing, on="listing_id", how="left",
              indicator=True, suffixes=('', '_y'))

changed:

1. insert to rental price history
2. update old one (fingerprint and last_updated)


In [44]:
changed = df[(df['fingerprint'] != df['fingerprint_y']) & (df['_merge'] == 'both')][[
    'listing_id', 'fingerprint', 'scraped_on', 'fingerprint_y', 'last_updated_y', '_merge']]
changed

,listing_id,fingerprint,scraped_on,fingerprint_y,last_updated_y,_merge
128,o3z2zYSPEvLYrVbx9HHG8k,o3z2zYSPEvLYrVbx9HHG8k-3350,2024-01-26,o3z2zYSPEvLYrVbx9HHG8k-3500,2024-01-25,both
230,PH6yTntn3Q5W6Yq4F2syGk,PH6yTntn3Q5W6Yq4F2syGk-5500,2024-01-26,PH6yTntn3Q5W6Yq4F2syGk-6000,2024-01-25,both
255,VKRV8AgVxRiXHjFmyesfsw,VKRV8AgVxRiXHjFmyesfsw-4900,2024-01-26,VKRV8AgVxRiXHjFmyesfsw-4500,2024-01-25,both
350,xzvUEgPBEKCnngo5Z2dm2G,xzvUEgPBEKCnngo5Z2dm2G-4000,2024-01-26,xzvUEgPBEKCnngo5Z2dm2G-4200,2024-01-25,both
408,8Wmzjje69dFY3xTuVs4hdb,8Wmzjje69dFY3xTuVs4hdb-3200,2024-01-26,8Wmzjje69dFY3xTuVs4hdb-3300,2024-01-25,both
414,NieAR22v4R7wD9wLDeJQd5,NieAR22v4R7wD9wLDeJQd5-4600,2024-01-26,NieAR22v4R7wD9wLDeJQd5-5300,2024-01-25,both
440,9RhKDgnA7BzkZGJHFYDSAb,9RhKDgnA7BzkZGJHFYDSAb-3700,2024-01-26,9RhKDgnA7BzkZGJHFYDSAb-3900,2024-01-25,both
490,oPRmxm3Z6cZDB92cPzrK6z,oPRmxm3Z6cZDB92cPzrK6z-4300,2024-01-26,oPRmxm3Z6cZDB92cPzrK6z-4350,2024-01-25,both


In [45]:
cdc = changed[['listing_id', 'fingerprint', 'last_updated_y']]
cdc['price'] = cdc['fingerprint'].apply(lambda x: int(x.split("-")[1]))
cdc.rename(columns={'last_updated_y': 'timestamp'}, inplace=True)
cdc = cdc[RENTAL_PRICE_HISTORY_COLS]
cdc

C:\Users\kaise\AppData\Local\Temp\ipykernel_6324\4040866765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc['price'] = cdc['fingerprint'].apply(lambda x: int(x.split("-")[1]))
C:\Users\kaise\AppData\Local\Temp\ipykernel_6324\4040866765.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc.rename(columns={'last_updated_y': 'timestamp'}, inplace=True)


,listing_id,price,timestamp
128,o3z2zYSPEvLYrVbx9HHG8k,3350,2024-01-25
230,PH6yTntn3Q5W6Yq4F2syGk,5500,2024-01-25
255,VKRV8AgVxRiXHjFmyesfsw,4900,2024-01-25
350,xzvUEgPBEKCnngo5Z2dm2G,4000,2024-01-25
408,8Wmzjje69dFY3xTuVs4hdb,3200,2024-01-25
414,NieAR22v4R7wD9wLDeJQd5,4600,2024-01-25
440,9RhKDgnA7BzkZGJHFYDSAb,3700,2024-01-25
490,oPRmxm3Z6cZDB92cPzrK6z,4300,2024-01-25


In [46]:
changed = changed[['listing_id', 'fingerprint', 'scraped_on']]
changed['price'] = changed['fingerprint'].apply(lambda x: int(x.split("-")[1]))
changed.rename(columns={'scraped_on': 'last_updated'}, inplace=True)
changed

C:\Users\kaise\AppData\Local\Temp\ipykernel_6324\1024784535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed['price'] = changed['fingerprint'].apply(lambda x: int(x.split("-")[1]))
C:\Users\kaise\AppData\Local\Temp\ipykernel_6324\1024784535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed.rename(columns={'scraped_on': 'last_updated'}, inplace=True)


,listing_id,fingerprint,last_updated,price
128,o3z2zYSPEvLYrVbx9HHG8k,o3z2zYSPEvLYrVbx9HHG8k-3350,2024-01-26,3350
230,PH6yTntn3Q5W6Yq4F2syGk,PH6yTntn3Q5W6Yq4F2syGk-5500,2024-01-26,5500
255,VKRV8AgVxRiXHjFmyesfsw,VKRV8AgVxRiXHjFmyesfsw-4900,2024-01-26,4900
350,xzvUEgPBEKCnngo5Z2dm2G,xzvUEgPBEKCnngo5Z2dm2G-4000,2024-01-26,4000
408,8Wmzjje69dFY3xTuVs4hdb,8Wmzjje69dFY3xTuVs4hdb-3200,2024-01-26,3200
414,NieAR22v4R7wD9wLDeJQd5,NieAR22v4R7wD9wLDeJQd5-4600,2024-01-26,4600
440,9RhKDgnA7BzkZGJHFYDSAb,9RhKDgnA7BzkZGJHFYDSAb-3700,2024-01-26,3700
490,oPRmxm3Z6cZDB92cPzrK6z,oPRmxm3Z6cZDB92cPzrK6z-4300,2024-01-26,4300


In [47]:
db.insert_df("rental_price_history", cdc)

In [48]:
db.update_table("property_listing", "listing_id", [
                "price", "fingerprint", "last_updated"], changed)

UPDATE tmp SET price = '3350', fingerprint = 'o3z2zYSPEvLYrVbx9HHG8k-3350', last_updated = '2024-01-26 00:00:00' WHERE listing_id = 'o3z2zYSPEvLYrVbx9HHG8k';UPDATE tmp SET price = '5500', fingerprint = 'PH6yTntn3Q5W6Yq4F2syGk-5500', last_updated = '2024-01-26 00:00:00' WHERE listing_id = 'PH6yTntn3Q5W6Yq4F2syGk';UPDATE tmp SET price = '4900', fingerprint = 'VKRV8AgVxRiXHjFmyesfsw-4900', last_updated = '2024-01-26 00:00:00' WHERE listing_id = 'VKRV8AgVxRiXHjFmyesfsw';UPDATE tmp SET price = '4000', fingerprint = 'xzvUEgPBEKCnngo5Z2dm2G-4000', last_updated = '2024-01-26 00:00:00' WHERE listing_id = 'xzvUEgPBEKCnngo5Z2dm2G';UPDATE tmp SET price = '3200', fingerprint = '8Wmzjje69dFY3xTuVs4hdb-3200', last_updated = '2024-01-26 00:00:00' WHERE listing_id = '8Wmzjje69dFY3xTuVs4hdb';UPDATE tmp SET price = '4600', fingerprint = 'NieAR22v4R7wD9wLDeJQd5-4600', last_updated = '2024-01-26 00:00:00' WHERE listing_id = 'NieAR22v4R7wD9wLDeJQd5';UPDATE tmp SET price = '3700', fingerprint = '9RhKDgnA7Bzk

In [49]:
new = df[df['_merge'] == 'left_only'][PROPERTY_LISTING_COLS]
new

,listing_id,property_name,district,price,bedroom,bathroom,dimensions,address,price_per_sqft,floor_level,...,distance_to_sch_in_m,nearest_mall,distance_to_mall_in_m,is_whole_unit,has_pool,has_gym,fingerprint,source,scraped_on,last_updated
3,MWASUKzWmFZWRKsnERisRK,Studio Condo in V On Shenton,Boat Quay / Raffles Place / Marina,3388,1,1,474,5A Shenton Way,7.0,Low,...,1040.306,Square 2,221.648,True,True,True,MWASUKzWmFZWRKsnERisRK-3388,ninety_nine,2024-01-26,2024-01-26
4,GsmAMQH99u3HkSxnAzaTQV,4 Bed Condo in The Sail @ Marina Bay,Boat Quay / Raffles Place / Marina,11999,4,3,2077,2 Marina Boulevard,5.0,High,...,1529.281,Marina Bay Link Mall,183.709,True,True,True,GsmAMQH99u3HkSxnAzaTQV-11999,ninety_nine,2024-01-26,2024-01-26
14,RWzHp3mHHQGEZcEMrmo2Br,Common Room in People's Park Centre,Boat Quay / Raffles Place / Marina,900,1,0,120,101A Upper Cross Street,7.0,NaN,...,1098.642,People's Park Centre,0.030,False,False,False,RWzHp3mHHQGEZcEMrmo2Br-900,ninety_nine,2024-01-26,2024-01-26
16,9YzYYyo4fuDrDGCBVyPvLX,1 Bed Condo in UIC Building,Boat Quay / Raffles Place / Marina,5200,1,1,743,5 Shenton Way,7.0,High,...,1096.227,Square 2,149.942,True,False,False,9YzYYyo4fuDrDGCBVyPvLX-5200,ninety_nine,2024-01-26,2024-01-26
19,feEEnZSjq5TZeptDikitTA,3 Bed Condo in Marina Bay Residences,Boat Quay / Raffles Place / Marina,14000,3,3,1625,18 Marina Boulevard,8.0,NaN,...,1732.859,Marina Bay Link Mall,107.281,True,True,True,feEEnZSjq5TZeptDikitTA-14000,ninety_nine,2024-01-26,2024-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,CbcADNxzdUqgXq3ht79EHY,2 Bed Condo in Riverbank @ Fernvale,Seletar / Yio Chu Kang,3400,2,1,900,17 Fernvale Close,3.0,High,...,684.040,The Seletar Mall,654.745,True,True,True,CbcADNxzdUqgXq3ht79EHY-3400,ninety_nine,2024-01-26,2024-01-26
822,DatwPLwMQrcwsmKzRSHCNe,4 Bed House on Alana,Seletar / Yio Chu Kang,8900,4,4,3488,Sunrise Terrace,2.0,NaN,...,1654.839,Greenwich V,1525.216,True,True,False,DatwPLwMQrcwsmKzRSHCNe-8900,ninety_nine,2024-01-26,2024-01-26
827,EgTLH9sz9SZJKZHBeghZqG,3 Bed Exec Condo in The Topiary,Seletar / Yio Chu Kang,4500,3,3,1055,17 Fernvale Lane,4.0,NaN,...,313.276,Greenwich V,340.564,True,True,True,EgTLH9sz9SZJKZHBeghZqG-4500,ninety_nine,2024-01-26,2024-01-26
830,c3crMegeUi6mVAgnqyHbTf,Common Room in 414B Fernvale Link,Seletar / Yio Chu Kang,550,1,0,160,414B Fernvale Link,3.0,NaN,...,571.050,The Seletar Mall,415.867,False,False,False,c3crMegeUi6mVAgnqyHbTf-550,ninety_nine,2024-01-26,2024-01-26


In [ ]:
db.insert_df("property_listing", new)

In [50]:
db.close()